In [2]:
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama

In [3]:
# loader_pdf = PyMuPDFLoader("/Users/silalertbanjongngam/Desktop/AI_learn/NAIST_GUIDEBOOK.pdf")
# pages_pdf = loader_pdf.load()
# for i in range(len(pages_pdf)):
#     pages_pdf[i].page_content = ' '.join(pages_pdf[i].page_content.split())  # Remove line breaks

# pages_pdf


from langchain_community.document_loaders import PyMuPDFLoader

loader_pdf = PyMuPDFLoader("/Users/silalertbanjongngam/Desktop/ttb-chatbot/PrinciplesForManagementofCreditRisk.pdf")
pages_pdf = loader_pdf.load()

# Define the new metadata you want to use
new_metadata = {
    "name": "Credit Risk Management Guidebook 2024",
    "description": "comprehensive guide on principles for managing credit risk.",
}

for i in range(len(pages_pdf)):
    # 1. Clean the content (your original code)
    pages_pdf[i].page_content = ' '.join(pages_pdf[i].page_content.split())  # Remove line breaks
    
    # 2. OVERWRITE the metadata
    # We use .copy() to ensure each page gets its own dictionary instance
    pages_pdf[i].metadata = new_metadata.copy()

# Check the result
print(pages_pdf)

[Document(metadata={'name': 'Credit Risk Management Guidebook 2024', 'description': 'comprehensive guide on principles for managing credit risk.'}, page_content='Credit risk management 1 Principles for the Management of Credit Risk I. Introduction 1. While financial institutions have faced difficulties over the years for a multitude of reasons, the major cause of serious banking problems continues to be directly related to lax credit standards for borrowers and counterparties, poor portfolio risk management, or a lack of attention to changes in economic or other circumstances that can lead to a deterioration in the credit standing of a bank’s counterparties. This experience is common in both G-10 and non-G-10 countries. 2. Credit risk is most simply defined as the potential that a bank borrower or counterparty will fail to meet its obligations in accordance with agreed terms. The goal of credit risk management is to maximise a bank’s risk-adjusted rate of return by maintaining credit r

In [6]:
from langchain_text_splitters.character import CharacterTextSplitter
char_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=300,
    chunk_overlap=50,
)

In [7]:
pages_char_split = char_splitter.split_documents(pages_pdf)

Created a chunk of size 415, which is longer than the specified 300
Created a chunk of size 350, which is longer than the specified 300
Created a chunk of size 344, which is longer than the specified 300
Created a chunk of size 388, which is longer than the specified 300
Created a chunk of size 307, which is longer than the specified 300
Created a chunk of size 973, which is longer than the specified 300
Created a chunk of size 315, which is longer than the specified 300
Created a chunk of size 316, which is longer than the specified 300
Created a chunk of size 461, which is longer than the specified 300
Created a chunk of size 347, which is longer than the specified 300
Created a chunk of size 429, which is longer than the specified 300
Created a chunk of size 397, which is longer than the specified 300
Created a chunk of size 308, which is longer than the specified 300
Created a chunk of size 312, which is longer than the specified 300
Created a chunk of size 332, which is longer tha

In [8]:
embed = OllamaEmbeddings(model="mxbai-embed-large:latest")

In [9]:
vector1 = embed.embed_query(pages_char_split[1].page_content)

In [12]:
from langchain_community.vectorstores import Chroma
vector_store = Chroma.from_documents(
    documents=pages_char_split,
    embedding=embed,
    persist_directory = "/Users/silalertbanjongngam/Desktop/ttb-chatbot/credit_risk_management_guidebook_vectorstore"
)
vectorstore_from_directory = Chroma(persist_directory = "/Users/silalertbanjongngam/Desktop/ttb-chatbot/credit_risk_management_guidebook_vectorstore", 
                                    embedding_function = embed)



/var/folders/l1/dlc_jzs90vj3g01l0jr_866w0000gn/T/ipykernel_37052/3874778851.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore_from_directory = Chroma(persist_directory = "/Users/silalertbanjongngam/Desktop/ttb-chatbot/credit_risk_management_guidebook_vectorstore",


In [13]:
vectorstore_from_directory.get()

{'ids': ['da7d34b7-4d2d-4b16-a87d-473959e7c698',
  'd214635e-91e9-4e5d-b332-0d69533340a9',
  'b916e312-54e4-41a9-9c6d-4a46e28c197e',
  'ce2b687f-826b-4d06-a4c9-d678ee18dfac',
  '96d09b8f-ba96-4321-860c-595cb3f4f741',
  '4f78d3c0-adc4-43c4-a7d9-a91c5ca1a6ab',
  'b45a4958-4f55-4299-a42e-9b2a8f5ac672',
  '7800f9d1-b05b-4ccc-ac75-58bda649686c',
  '52a51375-2858-4942-928b-9f6bb6e7d7c1',
  '5279ad83-0679-4685-9364-04fdb9cd3b4f',
  '132b3d19-a8cf-47b9-a8e1-9ff587a392f0',
  '5a57d7a5-207e-410d-824b-741c25a669be',
  '656f1871-04af-453a-b1ea-49889382547c',
  '1b896b8c-5de0-458c-9a20-baf52d52668d',
  'b9341dd1-7232-4011-adf9-931ffedddeb8',
  '98a100e7-aadf-4655-806e-305efcc9f9c3',
  '5ba2635e-d4c5-4cc7-ae73-9fed735cc20c',
  '9e267c8a-39a2-4a69-8af1-a53e369c3db4',
  '921a4ff9-3961-4ee7-b9ce-1489f0ed23dc',
  'd601aa51-bcf6-45a9-a46e-3b6713a2c889',
  '6e5351f0-6bdc-4fbf-b77c-d84d0e386cb6',
  '09863b78-eb52-44df-a682-22803c05d173',
  'f1c2ec3f-a87b-4cfd-80c9-f0c0dea1f38e',
  'a1731976-e0a4-4340-906c-

In [25]:
# 1. Set your desired threshold (0 to 1)
# Note: For Chroma with Cosine Similarity, 1 is perfect match, 0 is no match.
threshold = 0.7 
question = "What about credit risk mangement"
# 2. Use the method that returns (Document, Score) tuples
results_with_scores = vectorstore_from_directory.similarity_search_with_relevance_scores(
    query=question, 
    k=5
)

# 3. Filter the list based on the score
retrieved_docs = [doc for doc, score in results_with_scores if score >= threshold]

# Optional: Print scores to see what's happening
for doc, score in results_with_scores:
    print(f"Score: {score:.4f} | Content: {doc.page_content[:50]}...")

Score: 0.7820 | Content: The goal of credit risk management is to maximise ...
Score: 0.7554 | Content: 75. The goal of credit risk management is to maint...
Score: 0.7449 | Content: Banks should also consider the relationships betwe...
Score: 0.7199 | Content: Credit risk management 1 Principles for the Manage...
Score: 0.7100 | Content: Credit risk management 7 Each bank should develop ...


In [ ]:
from typing import Optional, Literal, Dict, Any
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langgraph.graph import START, END, StateGraph
from langchain_community.chat_models import ChatOllama
from datetime import datetime
from langchain_community.vectorstores import Chroma
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader

# 1. Configuration
llm = ChatOllama(model="gemma3:4b", temperature=0.2, max_completion_tokens=50)
embed = OllamaEmbeddings(model="mxbai-embed-large:latest")
# 1. Set your desired threshold (0 to 1)
# Note: For Chroma with Cosine Similarity, 1 is perfect match, 0 is no match.
vectorstore_from_directory = Chroma(persist_directory = "/Users/silalertbanjongngam/Desktop/ttb-chatbot/credit_risk_management_guidebook_vectorstore", 
                                    embedding_function = embed)


class GraphState(BaseModel):
    user_input: str
    decision: Optional[Literal["สินชื่อ", "เปิดบัญชีอย่างไร", "ยอดเงินไม่เข้า", "สแกนจ่ายไม่ได้", "unknown"]] = Field(default=None)
    final_output: Optional[str] = Field(default=None)
    raw_metadata: Dict[str, Any] = Field(default_factory=dict)

def retrieve_relevant_docs(query: str, threshold: float):

    # 1. Use the method that returns both the Doc and the Score
    # Note: 'k' here is the initial pool to check before filtering
    results_with_scores = vectorstore_from_directory.similarity_search_with_relevance_scores(
        query, 
        k=5
    )

    # 2. Filter based on your threshold
    # results_with_scores is a list of tuples: [(Document, score), ...]
    filtered_docs = [
        doc for doc, score in results_with_scores 
        if score >= threshold
    ]

    return filtered_docs

def validator(state: GraphState):
    """Hybrid Validator: String matching for buttons (0 tokens), LLM for text."""
    user_input = state.user_input.strip().lower()
    # Fast Path: Check if input exactly matches button options
    valid_options = ["สินชื่อ", "เปิดบัญชีอย่างไร", "ยอดเงินไม่เข้า", "สแกนจ่ายไม่ได้"]
    if user_input in valid_options:
        return {
            "decision": user_input, 
            "raw_metadata": {"method": "string_match", "tokens": 0, "time": datetime.now().isoformat()} 
        }
    return {"decision": "unknown"}

def option1_node(state: GraphState):
    response = (PromptTemplate.from_template("Tell me about strawberry in 10 words") | llm).invoke({})
    return {"final_output": response.content, "raw_metadata": response.response_metadata}

def option2_node(state: GraphState):
    response = (PromptTemplate.from_template("Tell me about Apple in 10 words") | llm).invoke({})
    return {"final_output": response.content, "raw_metadata": response.response_metadata}

def option3_node(state: GraphState):
    response = (PromptTemplate.from_template("Tell me about Orange in 10 words") | llm).invoke({})
    return {"final_output": response.content, "raw_metadata": response.response_metadata}

def option4_node(state: GraphState):
    response = (PromptTemplate.from_template("Tell me about Grape in 10 words") | llm).invoke({})
    return {"final_output": response.content, "raw_metadata": response.response_metadata}
#-------------------------------------------
# def unknown_node(state: GraphState):
#     """Handles free-text inquiry using the model."""
#     prompt = PromptTemplate.from_template("You are a helpful AI assistant. Answer this: {user_input} in very brief to the point.***used same language as user***")
#     chain = prompt | llm
#     # We MUST pass the user_input here so the model knows what to answer
#     response = chain.invoke({"user_input": state.user_input})
#     return {"final_output": response.content, "raw_metadata": response.response_metadata}
#-------------------------------------------

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def unknown_node(state: GraphState):
    """Handles free-text inquiry using RAG with threshold fallback logic."""
    user_input = state.user_input
    th = 0.75
    
    # 1. Try High Threshold (0.8)
    rag_results = retrieve_relevant_docs(user_input, threshold=th)
    
    # Check if we got results
    if len(rag_results) > 0:
        context_text = rag_results[0].page_content
        template = """You are an AI assistant for TTB bank. Answer the user based ONLY on context.
        Context: {context}
        Question: {question}
        Keep it professional and use the same language as the user."""
        
        prompt = PromptTemplate.from_template(template)
        chain = ({"context": lambda x: context_text, "question": RunnablePassthrough()} 
                 | prompt | llm | StrOutputParser())
        
        answer = chain.invoke(user_input)
        return {"final_output": answer}

    # 2. Try Lower Threshold (0.6)
    low_th = th - 0.15
    rag_results_low = retrieve_relevant_docs(user_input, threshold=low_th)
    
    if len(rag_results_low) > 0:
        context_text = rag_results_low[0].page_content
        template = """You are an AI assistant for TTB bank. 
        Create a new recommended question by adding on this context.
        Context: {context}
        User Query: {question}
        Keep it concise and use the same language as the user."""
        
        prompt = PromptTemplate.from_template(template)
        chain = ({"context": lambda x: context_text, "question": RunnablePassthrough()} 
                 | prompt | llm | StrOutputParser())
        
        recommendation = chain.invoke(user_input)
        return {"final_output": recommendation}

    # 3. Default Fallback (No Context Found)
    else:
        fallback_prompt = PromptTemplate.from_template(
            'You are a helpful AI assistant. Say: "I don\'t have that information about {user_input}" '
            'briefly and in the same language as the user.'
        )
        chain = fallback_prompt | llm | StrOutputParser()
        answer = chain.invoke({"user_input": user_input})
        return {"final_output": answer}



# 4. Router Logic
def router(state: GraphState):
    """Maps the decision string to the actual node name."""
    if state.decision in ["สินชื่อ", "เปิดบัญชีอย่างไร", "ยอดเงินไม่เข้า", "สแกนจ่ายไม่ได้"]:
        return state.decision
    return "unknown"

# 5. Graph Construction
builder = StateGraph(GraphState)

builder.add_node("validator", validator)
builder.add_node("option1_node", option1_node)
builder.add_node("option2_node", option2_node)
builder.add_node("option3_node", option3_node)
builder.add_node("option4_node", option4_node)
builder.add_node("unknown_node", unknown_node)

builder.add_edge(START, "validator")
builder.add_conditional_edges("validator", router, {
    "สินชื่อ": "option1_node",
    "เปิดบัญชีอย่างไร": "option2_node",
    "ยอดเงินไม่เข้า": "option3_node",
    "สแกนจ่ายไม่ได้": "option4_node",
    "unknown": "unknown_node"
})


builder.add_edge("option1_node", END)
builder.add_edge("option2_node", END)
builder.add_edge("option3_node", END)
builder.add_edge("option4_node", END)
builder.add_edge("unknown_node", END)

# Export for FastAPI
# graph = builder.compile()